In [7]:
#!pip install geopandas
#!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 37.6 MB/s eta 0:00:00:00:0100:01


In [8]:
import pandas as pd
import geopandas as gpd
import duckdb 

In [63]:
duckdb.sql('INSTALL httpfs; LOAD httpfs ;INSTALL spatial; LOAD spatial;')

In [109]:
#url = "http://minio.lab.sspcloud.fr/h529p3/data/Filosofi2017_carreaux_200m_met.parquet"
#url = "http://minio.lab.sspcloud.fr/h529p3/data/F*.parquet"
#url = ""
#url = "http://minio.lab.sspcloud.fr/h529p3/data/Filosofi2017_carreaux_200m_partition/bigsquare=662500-6862500/d68e6f93f32447deb1c989d91f737353-0.parquet"
#url = "http://minio.lab.sspcloud.fr/h529p3/data/Filosofi2017_carreaux_200m_partition/*/*.parquet"
#url = "Filosofi2017_carreaux_200m_met.parquet"
#url = 's3://h529p3/data/Filosofi2017_carreaux_200m_met.parquet?s3_endpoint=minio.lab.sspcloud.fr&s3_region=us-east-1'
url = "s3://h529p3/data/Filosofi2017_carreaux_200m_partition/bigsquare=662500-6862500/*.parquet?s3_endpoint=minio.lab.sspcloud.fr&s3_region=us-east-1"
print(url)

s3://h529p3/data/Filosofi2017_carreaux_200m_partition/bigsquare=662500-6862500/*.parquet?s3_endpoint=minio.lab.sspcloud.fr&s3_region=us-east-1


In [110]:
duckdb.sql(f'''
  CREATE OR REPLACE VIEW carreaux
  AS SELECT * FROM read_parquet("{url}", hive_partitioning = 1)
'''
)

IOException: IO Error: HTTP GET error on '/?encoding-type=url&list-type=2&prefix=data%2FFilosofi2017_carreaux_200m_partition%2Fbigsquare%3D662500-6862500%2F' (HTTP 403)

In [108]:
carreaux_test = duckdb.sql(
  "SELECT * FROM carreaux"
  ).to_df()


Requetage des carreaux à 1km de notre dame

In [91]:
latitude = 48.853
longitude = 2.35
rayon = 1000

In [92]:
test =  duckdb.sql(
f'''select * from carreaux, (SELECT ST_BUFFER(ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154'),{rayon}) AS hubblo) '''\
"where st_intersects(ST_GEOMFROMWKB(carreaux.geometry), hubblo) and bigsquare=='662500-6862500'"
)


BinderException: Binder Error: Referenced column "bigsquare" not found in FROM clause!
Candidate bindings: "carreaux.Groupe"

In [90]:
test

InvalidInputException: Invalid Input Error: Attempting to execute an unsuccessful or closed pending query result
Error: Binder Error: Referenced column "bigsquare" not found in FROM clause!
Candidate bindings: "carreaux.Groupe"

In [87]:
duckdb.__version__

'0.9.2'